# 영어를 한 문장씩 출력해 주는 AI 프로그램 만들기

## OpenAI API 키 불러오기

In [1]:
import os
import openai

# 환경변수에서 OpenAI API 키를 읽어옴.
openai_api_key = os.getenv('OPENAI_API_KEY')

# API 키가 환경변수에 설정되어 있는지 확인.
if not openai_api_key:
    raise ValueError("환경변수 'OPENAI_API_KEY'가 설정되지 않았습니다.")

# OpenAI API 키를 설정.
openai.api_key = openai_api_key

# OpenAI 클라이언트를 생성.
client = openai.OpenAI()

## 레이아웃 구성하기 (그라디오)

In [2]:
import gradio as gr
with gr.Blocks() as app:
    with gr.Tab("영어 학습봇"): # 탭
        with gr.Column(): 
            gr.Markdown() # 제목
        with gr.Row():
            with gr.Column():
                with gr.Column():
                    gr.Dropdown() #1
                    gr.Slider() #2
                    gr.Dropdown() #3
                    gr.Button() #4
            gr.Textbox() #5

        with gr.Row():
            gr.Textbox() #6
            gr.Button() #7
app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## 레이아웃 다듬기, 컴포넌트 추가하기

In [3]:
import gradio as gr
with gr.Blocks() as app:
    with gr.Tab("영어 학습봇"): # 탭
        with gr.Column(): 
            gr.Markdown(
                value="""
                # <center>영어 학습봇</center>
                <center>인공지능 비서 DUDE 입니다.
                영어 학습을 위한 영어 한 문장을 생성합니다.</center>
                """
            ) # 제목
        with gr.Row():
            with gr.Column():
                with gr.Column():
                    en_type = gr.Dropdown( #1
                        label="타입 선택",
                        choices=["일반회화", "비즈니스", "명언", "시사"],
                        value = "일반회화",
                        interactive=True   
                    ) 
                    gr.Slider( #2
                        label="속도",
                        info="숫자가 높을수록 속도 빠름",
                        minimum=0,
                        maximum=2,
                        step=0.2,
                        value=1,
                        interactive=True                        
                    ) 
                    en_voice = gr.Dropdown( #3
                        label="목소리 선택",
                        choices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                        value = "alloy",
                        interactive=True                        
                    ) 
                    cb_submit_btn=gr.Button(
                        value="음성 듣기",
                        scale=1,
                        visible="primary",
                        icon="https://cdn-icons-png.flaticon.com/128/12439/12439334.png" # 재생버튼
                    ) #4
            gr.Textbox( #5
                container=False,
                placeholder="AI와 함께 영어 공부를!",
                lines=3,
                scale=4,
                show_copy_button=True # 내용 복사 기능
            ) 

        with gr.Row():
            gr.Textbox( #6
                label="",
                placeholder="AI가 생성한 문장을 해석 및 설명합니다.",
                lines=5,
                max_lines=20,
                show_copy_button=True # 내용 복사 기능                
            ) 
            en_create=gr.Button( #7
                value="🔄️생성하기",
                scale=0.3,
                visible='primary'
                ) 
app.launch()

C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\gradio\components\base.py:193: UserWarning: 'scale' value should be an integer. Using 0.3 will cause issues.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## 기능 설정 (함수 정의)

In [7]:
import pyttsx3 # 텍스트 음성

# 문장을 생성하는 함수
def generate_sentence(en_type, speed, en_voice):
    # TODO: OpenAI API를 사용하여 문장 생성 로직 추가
    # 선택한 카테고리로 문장 생성
    prompt =  f"Generate a sentence in the {en_type} category." 
    try:
        response = client.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=60,
            n=1,
            stop=None,
            temperature=0.7
        )
        generated_sentence = response.choices[0].text.strip()
        return generated_sentence
    except Exception as e:
        return f"문장 생성 중 오류 발생: {e}"

# 문장 발음을 위한 함수 (placeholder)
def play_sentence(generated_sentence, speed, en_voice):
    # 텍스트 음성을 위한 pyttsx3 초기화
    engine = pyttsx3.init()

    # 속도 설정
    engine.setProperty('rate', 150 + (speed * 50)) # 기본 속도 조절(기본속도 = 150)

    # 목소리 설정
    voices = engine.getProperty('voices')
    voice_index = {"alloy":0, "echo":1, "fable":2, "onyx":3, "nova":4, "shimmer":5}
    if en_voice in voice_index:
        engine.setProperty('voice', voices[voice_index[en_voice]].id)

    # 문장 읽기
    engine.say(generated_sentence)
    engine.runAndWait()


# 문장 해석 및 설명 함수
def explain_sentence(sentence):
    # 문장을 해석하는 기본적인 설명
    explanation = f"The sentence '{sentence}' means that..."
    # 실제 문법적인 부분이나 의미를 추가적으로 설명할 수 있는 부분
    
    return explanation

In [6]:
!pip install pyttsx3

## 이벤트 리스너 추가

In [31]:
import os
import openai
import pyttsx3 # 텍스트 음성

# 환경변수에서 OpenAI API 키를 읽어옴.
openai_api_key = os.getenv('OPENAI_API_KEY')

# API 키가 환경변수에 설정되어 있는지 확인.
if not openai_api_key:
    raise ValueError("환경변수 'OPENAI_API_KEY'가 설정되지 않았습니다.")

# OpenAI API 키를 설정.
openai.api_key = openai_api_key

# OpenAI 클라이언트를 생성.
client = openai.OpenAI()


# 문장을 생성하는 함수
def generate_sentence(en_type, speed, en_voice):
    prompt = f"Generate an English sentence in the category of '{en_type}'."
    try:
        completion = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            max_tokens=60
        )
        
        generated_sentence = completion.choices[0].message.content
        return generated_sentence
    except Exception as e:
        return f"문장 생성 중 오류 발생: {e}"

# 문장 발음을 위한 함수
# 사용 가능한 목소리 목록 가져오기
def get_available_voices():
    # TTS 엔진 초기화
    engine = pyttsx3.init()

    # 사용 가능한 목소리 목록 가져오기
    voices = engine.getProperty('voices')
    voice_choices = []
    
    for voice in voices:
        voice_choices.append(voice.name)  # 목소리 이름을 리스트에 추가
        
    return voice_choices

# 음성 재생 함수
# 음성 파일 다운로드 없이 바로 재생될 수 있게 pyttsx3 엔진 사용
def play_sentence(generated_sentence, speed, en_voice):
    # 텍스트 음성을 위한 pyttsx3 초기화
    engine = pyttsx3.init()

    # 속도 설정
    engine.setProperty('rate', 150 + (speed * 50)) # 기본 속도 조절(기본속도 = 150)

    # 목소리 설정
    voices = engine.getProperty('voices')
    for voice in voices:
        if voice.name == en_voice:
            engine.setProperty('voice', voice.id)
   
    # 문장 읽기
    engine.say(generated_sentence)
    engine.runAndWait()


# 문장 해석 및 설명 함수
def explain_sentence(sentence):
    # 문장을 해석하는 기본적인 설명
    # 한국어로 출력되게 하기 위해 한국어로 프롬프트 작성
    explanation_prompt =  f"문장 '{sentence}'을(를) 한국어로 번역하고 해당 문장에 사용된 문법이나 단어를 설명해 주세요."
    try:
        completion = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": explanation_prompt
                }
            ],
            max_tokens=1000 # 보다 긴 설명을 위한 토큰 수 증가
        )
        
        generated_explanation = completion.choices[0].message.content # 생성된 설명 받
        return generated_explanation
    except Exception as e:
        return f"문장 생성 중 오류 발생: {e}"

import gradio as gr
with gr.Blocks() as app:
    with gr.Tab("영어 학습봇"): # 탭
        with gr.Column(): 
            gr.Markdown(
                value="""
                # <center>영어 학습봇</center>
                <center>인공지능 비서 DUDE 입니다.
                영어 학습을 위한 영어 한 문장을 생성합니다.</center>
                """
            ) # 제목
        with gr.Row():
            with gr.Column():
                with gr.Column():
                    en_type = gr.Dropdown( #1
                        label="타입 선택",
                        choices=["일반회화", "비즈니스", "명언", "시사"],
                        value = "일반회화",
                        interactive=True   
                    ) 
                    speed = gr.Slider( #2
                        label="속도",
                        info="숫자가 높을수록 속도 빠름",
                        minimum=0,
                        maximum=2,
                        step=0.2,
                        value=1,
                        interactive=True                        
                    )
                    
                    # 사용 가능한 목소리 리스트 가져오기
                    voice_choices = get_available_voices()
                    
                    en_voice = gr.Dropdown( #3
                        label="목소리 선택",
                        choices = voice_choices,
                        value = "alloy",
                        interactive=True                        
                    ) 
                    play_btn=gr.Button(
                        value="음성 듣기",
                        scale=1,
                        visible="primary",
                        icon="https://cdn-icons-png.flaticon.com/128/12439/12439334.png" # 재생버튼
                    ) #4
            
            generated_sentence_box = gr.Textbox( #5
                container=False,
                placeholder="AI와 함께 영어 공부를!",
                lines=3,
                scale=4,
                show_copy_button=True # 내용 복사 기능
            ) 

        with gr.Row():
            explanation_box = gr.Textbox( #6
                label="",
                placeholder="AI가 생성한 문장을 해석 및 설명합니다.",
                lines=10,
                max_lines=100,
                show_copy_button=True # 내용 복사 기능                
            ) 
            en_create=gr.Button( #7
                value="🔄️생성하기",
                scale=0.3,
                visible='primary'
                ) 

        # 이벤트 리스너
        en_create.click(
            fn=generate_sentence,
            inputs=[en_type, speed, en_voice],
            outputs=generated_sentence_box
        )

        play_btn.click(
            fn=play_sentence,
            inputs=[generated_sentence_box, speed, en_voice],
            outputs=None
        )

        generated_sentence_box.change(
            fn=explain_sentence,
            inputs=generated_sentence_box,
            outputs=explanation_box
        )

            
app.launch()

C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\gradio\components\dropdown.py:188: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: alloy or set allow_custom_value=True.
  warnings.warn(
C:\Users\niceq\anaconda3\envs\ai_jun\lib\site-packages\gradio\components\base.py:193: UserWarning: 'scale' value should be an integer. Using 0.3 will cause issues.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.
